In [ ]:
import pandas as pd

iris = pd.read_csv("/content/iris.csv")

iris["x1"] = iris["sepal.length"]
iris["x2"] = iris["sepal.width"]
iris["x3"] = iris["petal.length"]
iris["x4"] = iris["petal.width"]
iris["y1"] = (iris["variety"] == "Setosa").astype(int)
iris["y2"] = (iris["variety"] == "Versicolor").astype(int)
iris["y3"] = (iris["variety"] == "Virginica").astype(int)

X = iris[["x1", "x2", "x3", "x4"]]

Y = iris[["y1", "y2", "y3"]]

X.join(Y)

,x1,x2,x3,x4,y1,y2,y3
0,5.1,3.5,1.4,0.2,1,0,0
1,4.9,3.0,1.4,0.2,1,0,0
2,4.7,3.2,1.3,0.2,1,0,0
3,4.6,3.1,1.5,0.2,1,0,0
4,5.0,3.6,1.4,0.2,1,0,0
...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0,0,1
146,6.3,2.5,5.0,1.9,0,0,1
147,6.5,3.0,5.2,2.0,0,0,1
148,6.2,3.4,5.4,2.3,0,0,1


In [ ]:
import keras # v2

# 1. Topología de la Red Neuronal 4 -> 100 -> 3
model = keras.Sequential([
    keras.Input(shape=(4,)),
    # keras.layers.Dense(100, activation="tanh", name="Hidden1"), # [-1, 1]
    keras.layers.Dense(100, activation="relu", name="Hidden2"), # [0, inf]
    keras.layers.Dense(3, activation="softmax", name="Output"), # [0, 1]
], name="Iris")

# 2. Compilar la Red Neuronal (Optimizador + Función de Pérdida)
model.compile(optimizer="sgd", loss="mse")

model.summary()

Model: "Iris"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Hidden2 (Dense)             (None, 100)               500       
                                                                 
 Output (Dense)              (None, 3)                 303       
                                                                 
Total params: 803 (3.14 KB)
Trainable params: 803 (3.14 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# 3. Fase de entrenamiento/ajuste (batch_size -> tamaño de las muestras | epochs -> épocas/#veces)
model.fit(X, Y, batch_size=150, epochs=1_000)

Epoch 1/1000
1/1 [==============================] - 0s 330ms/step - loss: 0.2268
Epoch 2/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.2243
Epoch 3/1000
1/1 [==============================] - 0s 8ms/step - loss: 0.2221
Epoch 4/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.2199
Epoch 5/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.2178
Epoch 6/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.2157
Epoch 7/1000
1/1 [==============================] - 0s 9ms/step - loss: 0.2137
Epoch 8/1000
1/1 [==============================] - 0s 12ms/step - loss: 0.2118
Epoch 9/1000
1/1 [==============================] - 0s 13ms/step - loss: 0.2099
Epoch 10/1000
1/1 [==============================] - 0s 12ms/step - loss: 0.2080
Epoch 11/1000
1/1 [==============================] - 0s 11ms/step - loss: 0.2062
Epoch 12/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.2044
Epoch 13/1000
1/1 [=====================

In [ ]:
import pandas as pd

Yp = pd.DataFrame(model.predict(X), columns=["y1p", "y2p", "y3p"])

Yp["y1p"] = (Yp["y1p"] >= 0.5).astype(int)
Yp["y2p"] = (Yp["y2p"] >= 0.5).astype(int)
Yp["y3p"] = (Yp["y3p"] >= 0.5).astype(int)

Yp

5/5 [==============================] - 0s 3ms/step


,y1p,y2p,y3p
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
145,0,0,1
146,0,0,1
147,0,0,1
148,0,0,1


In [ ]:
data1 = X.join(Y).join(Yp).copy()

data1

,x1,x2,x3,x4,y1,y2,y3,y1p,y2p,y3p
0,5.1,3.5,1.4,0.2,1,0,0,1,0,0
1,4.9,3.0,1.4,0.2,1,0,0,1,0,0
2,4.7,3.2,1.3,0.2,1,0,0,1,0,0
3,4.6,3.1,1.5,0.2,1,0,0,1,0,0
4,5.0,3.6,1.4,0.2,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0,0,1,0,0,1
146,6.3,2.5,5.0,1.9,0,0,1,0,0,1
147,6.5,3.0,5.2,2.0,0,0,1,0,0,1
148,6.2,3.4,5.4,2.3,0,0,1,0,0,1


In [ ]:
data1["F"] = (((data1["y1"] != data1["y1p"]).astype(int) + (data1["y2"] != data1["y2p"]).astype(int) + (data1["y3"] != data1["y3p"]).astype(int)) > 0).astype(int)
data1["C"] = 1 - data1["F"]

data1[["F", "C"]].describe()

,F,C
count,150.000000,150.000000
mean,0.060000,0.940000
std,0.238282,0.238282
min,0.000000,0.000000
25%,0.000000,1.000000
50%,0.000000,1.000000
75%,0.000000,1.000000
max,1.000000,1.000000
